In [3]:
# Import libraries untuk data cleaning
import pandas as pd
import numpy as np

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv

# --- Konfigurasi bulan Indonesia ---
bulan_id = {
    1: "Januari", 2: "Febuari", 3: "Maret", 4: "April",
    5: "Mei", 6: "Juni", 7: "Juli", 8: "Agustus",
    9: "September", 10: "Oktober", 11: "November", 12: "Desember"
}

# Rentang waktu scraping
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 3, 13)

headers = {"User-Agent": "Mozilla/5.0"}

# --- Fungsi scraping per hari ---
def scrape_per_hari(tahun, bulan, tanggal):
    nama_bulan = bulan_id[bulan]
    url = f"https://harga-emas.org/history-harga/{tahun}/{nama_bulan}/{tanggal:02d}/"
    try:
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.content, "html.parser")
        rows = soup.find_all("tr")

        if len(rows) < 12:
            return None

        # Ambil baris ounce dan gram berdasarkan asumsi posisi tetap
        oz_cells = rows[10].find_all("td")
        gr_cells = rows[11].find_all("td")

        harga_oz = oz_cells[1].text.strip()
        kurs_oz = oz_cells[2].text.strip()
        idr_oz = oz_cells[3].text.strip()

        harga_gr = gr_cells[1].text.strip()
        kurs_gr = gr_cells[2].text.strip()
        idr_gr = gr_cells[3].text.strip()

        # Ambil tanggal update
        update_raw = ""
        for row in rows:
            if "Update harga emas" in row.text:
                update_raw = row.text.strip()
                break

        update_tanggal = update_raw.split("Update harga emas :")[-1].split("Update kurs")[0].strip()
        tanggal_label = f"{tanggal:02d} {nama_bulan} {tahun}"

        return {
            "tanggal": tanggal_label,
            "harga_oz": harga_oz,
            "kurs_oz": kurs_oz,
            "idr_oz": idr_oz,
            "harga_gr": harga_gr,
            "kurs_gr": kurs_gr,
            "idr_gr": idr_gr,
            "update_asli": update_tanggal
        }
    except Exception as e:
        print(f"Gagal scraping {tanggal:02d}-{nama_bulan}-{tahun}: {e}")
        return None

# --- Loop utama ---
results = []
current = start_date
while current <= end_date:
    nama_bulan = bulan_id[current.month]
    print(f"Memproses {current.day:02d} {nama_bulan} {current.year}...")
    data = scrape_per_hari(current.year, current.month, current.day)
    if data:
        results.append(data)
    current += timedelta(days=1)

# --- Simpan hasil ke CSV ---
output_filename = "harga_emas_febuari_2020-2025.csv"
with open(output_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=results[0].keys())
    writer.writeheader()
    writer.writerows(results)

print(f"✅ Selesai menyimpan data ke {output_filename}")


Memproses 01 Januari 2020...
Memproses 02 Januari 2020...
Memproses 03 Januari 2020...
Memproses 02 Januari 2020...
Memproses 03 Januari 2020...
Memproses 04 Januari 2020...
Memproses 05 Januari 2020...
Memproses 04 Januari 2020...
Memproses 05 Januari 2020...
Memproses 06 Januari 2020...
Memproses 07 Januari 2020...
Memproses 06 Januari 2020...
Memproses 07 Januari 2020...
Memproses 08 Januari 2020...
Memproses 09 Januari 2020...
Memproses 08 Januari 2020...
Memproses 09 Januari 2020...
Memproses 10 Januari 2020...
Memproses 11 Januari 2020...
Memproses 10 Januari 2020...
Memproses 11 Januari 2020...
Memproses 12 Januari 2020...
Memproses 12 Januari 2020...
Memproses 13 Januari 2020...
Memproses 14 Januari 2020...
Memproses 13 Januari 2020...
Memproses 14 Januari 2020...
Memproses 15 Januari 2020...
Memproses 16 Januari 2020...
Memproses 15 Januari 2020...
Memproses 16 Januari 2020...
Memproses 17 Januari 2020...
Memproses 18 Januari 2020...
Memproses 17 Januari 2020...
Memproses 18 J

KeyboardInterrupt: 

---

# BAGIAN SCRAPING SELESAI

File `harga_emas_febuari_2020-2025.csv` telah dibuat. 
Sekarang lanjut ke proses **Data Cleaning**.

---

# Data Cleaning untuk Harga Emas

Notebook ini berisi proses cleaning data hasil scraping harga emas dari file `harga_emas_febuari_2020-2025.csv`

## Langkah-langkah Cleaning:
1. Load dan eksplorasi data
2. Cleaning kolom tanggal  
3. Cleaning kolom numerik (harga dan kurs)
4. Rename kolom untuk konsistensi
5. Final validation

In [2]:
import pandas as pd

# ===== 1. LOAD DAN EKSPLORASI DATA =====

# Load data dari CSV
df = pd.read_csv('harga_emas_febuari_2020-2025.csv')

print("=== INFORMASI DASAR DATASET ===")
print(f"Jumlah baris: {len(df)}")
print(f"Jumlah kolom: {len(df.columns)}")
print(f"Kolom: {list(df.columns)}")

print("\n=== PREVIEW DATA ===")
df.head()

=== INFORMASI DASAR DATASET ===
Jumlah baris: 1899
Jumlah kolom: 8
Kolom: ['tanggal', 'harga_oz', 'kurs_oz', 'idr_oz', 'harga_gr', 'kurs_gr', 'idr_gr', 'update_asli']

=== PREVIEW DATA ===


,tanggal,harga_oz,kurs_oz,idr_oz,harga_gr,kurs_gr,idr_gr,update_asli
0,01 Januari 2020,"1.517,24 (-1.107,25)","13.901,01 (-2.260,99)",21.091.168,"48,78","13.901,01","678.097 (-685.641,64)","01 Januari 2020, pukul 23:59"
1,02 Januari 2020,"1.526,12 (-1.098,37)","13.895,01 (-2.266,99)",21.205.453,"49,07","13.895,01","681.771 (-681.967,32)","02 Januari 2020, pukul 23:59"
2,03 Januari 2020,"1.548,99 (-1.075,50)","13.899,01 (-2.262,99)",21.529.427,"49,80","13.899,01","692.187 (-671.551,29)","03 Januari 2020, pukul 23:59"
3,04 Januari 2020,"1.552,24 (-1.072,25)","13.899,01 (-2.262,99)",21.574.599,"49,91","13.899,01","693.639 (-670.098,98)","04 Januari 2020, pukul 23:59"
4,05 Januari 2020,"1.552,24 (-1.072,25)","13.899,01 (-2.262,99)",21.574.599,"49,91","13.899,01","693.639 (-670.098,98)","05 Januari 2020, pukul 23:59"


In [3]:
# ===== EKSPLORASI LANJUTAN =====

print("=== TIPE DATA ===")
print(df.dtypes)

print("\n=== STATISTIK DESKRIPTIF ===")
print(df.describe(include='all'))

print("\n=== MISSING VALUES ===")
print(df.isna().sum())

print("\n=== CONTOH DATA MENTAH ===")
print("Sample tanggal:")
print(df['tanggal'].head(10).tolist())
print("\nSample harga:")
print(df['harga_oz'].head(5).tolist())

=== TIPE DATA ===
tanggal        object
harga_oz       object
kurs_oz        object
idr_oz         object
harga_gr       object
kurs_gr        object
idr_gr         object
update_asli    object
dtype: object

=== STATISTIK DESKRIPTIF ===
                tanggal            harga_oz              kurs_oz      idr_oz  \
count              1899                1899                 1899        1899   
unique             1899                1387                  859        1409   
top     01 Januari 2020  1.823,86 (-800,63)  15.731,00 (-431,00)  28.691.142   
freq                  1                 243                  245         243   

       harga_gr    kurs_gr                 idr_gr  \
count      1899       1899                   1899   
unique     1033        838                   1409   
top       58,64  15.731,00  922.442 (-441.296,83)   
freq        244        245                    243   

                         update_asli  
count                           1899  
unique           

## 2. CLEANING KOLOM TANGGAL

Kolom tanggal saat ini dalam format string "DD Bulan YYYY" (contoh: "01 Januari 2020").
Kita perlu mengkonversi ke format datetime untuk analisis yang lebih baik.

In [4]:
# Clean and change datatype tanggal

# ===== KONVERSI TANGGAL KE DATETIME =====

# Mapping bulan Indonesia ke angka
bulan_mapping = {
    "Januari": "01", "Febuari": "02", "Februari": "02", "Maret": "03", 
    "April": "04", "Mei": "05", "Juni": "06", "Juli": "07", 
    "Agustus": "08", "September": "09", "Oktober": "10", 
    "November": "11", "Desember": "12"
}

def konversi_tanggal(tanggal_str):
    """Konversi dari 'DD Bulan YYYY' ke 'YYYY-MM-DD'"""
    try:
        parts = tanggal_str.split()
        if len(parts) != 3:
            return None
        hari, bulan, tahun = parts
        bulan_num = bulan_mapping.get(bulan, "00")
        return f"{tahun}-{bulan_num}-{hari.zfill(2)}"
    except:
        return None

# Konversi tanggal
print("Memproses konversi tanggal...")
df['tanggal_formatted'] = df['tanggal'].apply(konversi_tanggal)
df['tanggal'] = pd.to_datetime(df['tanggal_formatted'], errors='coerce')

# Hapus kolom sementara
df.drop(columns=['tanggal_formatted'], inplace=True)

print("✅ Konversi tanggal selesai!")
print(f"Tanggal berhasil dikonversi: {df['tanggal'].notna().sum()}")
print(f"Tanggal gagal dikonversi: {df['tanggal'].isna().sum()}")

# Preview hasil
print("\n=== PREVIEW TANGGAL SETELAH KONVERSI ===")
df[['tanggal']].head(10)


Memproses konversi tanggal...
✅ Konversi tanggal selesai!
Tanggal berhasil dikonversi: 1899
Tanggal gagal dikonversi: 0

=== PREVIEW TANGGAL SETELAH KONVERSI ===


,tanggal
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05
5,2020-01-06
6,2020-01-07
7,2020-01-08
8,2020-01-09
9,2020-01-10


In [5]:
# ===== VALIDASI TANGGAL =====

print("=== VALIDASI HASIL KONVERSI TANGGAL ===")
print(f"Range tanggal: {df['tanggal'].min()} hingga {df['tanggal'].max()}")
print(f"Total hari: {len(df)}")

# Cek duplikasi tanggal
duplicates = df['tanggal'].value_counts()
if (duplicates > 1).any():
    print(f"\n⚠️  Ada {(duplicates > 1).sum()} tanggal yang duplikat:")
    print(duplicates[duplicates > 1].head())
else:
    print("\n✅ Tidak ada duplikasi tanggal")

=== VALIDASI HASIL KONVERSI TANGGAL ===
Range tanggal: 2020-01-01 00:00:00 hingga 2025-03-13 00:00:00
Total hari: 1899

✅ Tidak ada duplikasi tanggal


## 3. CLEANING DATA NUMERIK (HARGA & KURS)

Kolom harga dan kurs masih dalam format string dengan:
- Pemisah ribuan berupa titik (.)
- Desimal berupa koma (,)
- Ada nilai dalam kurung () yang perlu dihapus
- Perlu dikonversi ke tipe float

In [6]:
# ===== CLEANING DATA NUMERIK =====

def clean_numeric_column(series):
    """
    Fungsi untuk membersihkan kolom numerik:
    - Hapus teks dalam kurung ()
    - Hapus pemisah ribuan (titik)
    - Ganti koma desimal dengan titik
    - Konversi ke float
    """
    # Hapus teks dalam kurung
    cleaned = series.str.replace(r'\s*\(.*?\)', '', regex=True)
    
    # Hapus pemisah ribuan dan fix desimal
    cleaned = cleaned.str.replace('.', '', regex=False)
    cleaned = cleaned.str.replace(',', '.', regex=False)
    
    # Konversi ke float
    return pd.to_numeric(cleaned, errors='coerce')

# Daftar kolom yang perlu dibersihkan
numeric_columns = ['harga_oz', 'kurs_oz', 'idr_oz', 'harga_gr', 'kurs_gr', 'idr_gr']

print("=== CLEANING KOLOM NUMERIK ===")
for col in numeric_columns:
    print(f"Memproses kolom: {col}")
    
    # Show before
    print(f"  Contoh sebelum: {df[col].iloc[0]}")
    
    # Clean
    df[col] = clean_numeric_column(df[col])
    
    # Show after
    print(f"  Contoh sesudah: {df[col].iloc[0]}")
    print(f"  Missing values: {df[col].isna().sum()}")
    print()

print("✅ Cleaning kolom numerik selesai!")



=== CLEANING KOLOM NUMERIK ===
Memproses kolom: harga_oz
  Contoh sebelum: 1.517,24 (-1.107,25)
  Contoh sesudah: 1517.24
  Missing values: 0

Memproses kolom: kurs_oz
  Contoh sebelum: 13.901,01 (-2.260,99)
  Contoh sesudah: 13901.01
  Missing values: 0

Memproses kolom: idr_oz
  Contoh sebelum: 21.091.168
  Contoh sesudah: 21091168
  Missing values: 0

Memproses kolom: harga_gr
  Contoh sebelum: 48,78
  Contoh sesudah: 48.78
  Missing values: 0

Memproses kolom: kurs_gr
  Contoh sebelum: 13.901,01
  Contoh sesudah: 13901.01
  Missing values: 0

Memproses kolom: idr_gr
  Contoh sebelum: 678.097 (-685.641,64)
  Contoh sesudah: 678097
  Missing values: 0

✅ Cleaning kolom numerik selesai!


In [7]:
# ===== VALIDASI HASIL CLEANING NUMERIK =====

print("=== VALIDASI TIPE DATA ===")
print(df.dtypes)

print("\n=== STATISTIK DESKRIPTIF ===")
print(df.describe())

print("\n=== CONTOH DATA SETELAH CLEANING ===")
df.head()

=== VALIDASI TIPE DATA ===
tanggal        datetime64[ns]
harga_oz              float64
kurs_oz               float64
idr_oz                  int64
harga_gr              float64
kurs_gr               float64
idr_gr                  int64
update_asli            object
dtype: object

=== STATISTIK DESKRIPTIF ===
                   tanggal     harga_oz       kurs_oz        idr_oz  \
count                 1899  1899.000000   1899.000000  1.899000e+03   
mean   2022-08-07 00:00:00  1960.730679  15100.356261  2.974355e+07   
min    2020-01-01 00:00:00  1474.030000  13612.000000  2.106676e+07   
25%    2021-04-19 12:00:00  1783.395000  14373.010000  2.591320e+07   
50%    2022-08-07 00:00:00  1826.030000  15083.010000  2.783393e+07   
75%    2023-11-24 12:00:00  2004.550000  15731.000000  3.134428e+07   
max    2025-03-13 00:00:00  2981.490000  16741.010000  4.905446e+07   
std                    NaN   313.713540    752.493483  5.883284e+06   

          harga_gr       kurs_gr        idr_gr  


,tanggal,harga_oz,kurs_oz,idr_oz,harga_gr,kurs_gr,idr_gr,update_asli
0,2020-01-01,1517.24,13901.01,21091168,48.78,13901.01,678097,"01 Januari 2020, pukul 23:59"
1,2020-01-02,1526.12,13895.01,21205453,49.07,13895.01,681771,"02 Januari 2020, pukul 23:59"
2,2020-01-03,1548.99,13899.01,21529427,49.80,13899.01,692187,"03 Januari 2020, pukul 23:59"
3,2020-01-04,1552.24,13899.01,21574599,49.91,13899.01,693639,"04 Januari 2020, pukul 23:59"
4,2020-01-05,1552.24,13899.01,21574599,49.91,13899.01,693639,"05 Januari 2020, pukul 23:59"


In [8]:
# ===== HAPUS KOLOM YANG TIDAK DIPERLUKAN =====

# Hapus kolom update_asli karena tidak diperlukan untuk analisis
if 'update_asli' in df.columns:
    df = df.drop(columns=['update_asli'])
    print("✅ Kolom 'update_asli' telah dihapus")
else:
    print("ℹ️  Kolom 'update_asli' tidak ditemukan")

print(f"Kolom yang tersisa: {list(df.columns)}")


✅ Kolom 'update_asli' telah dihapus
Kolom yang tersisa: ['tanggal', 'harga_oz', 'kurs_oz', 'idr_oz', 'harga_gr', 'kurs_gr', 'idr_gr']


## 4. FINALISASI DATA

Tahap akhir cleaning:
- Hapus kolom yang tidak diperlukan
- Rename kolom untuk konsistensi
- Validasi final

In [9]:
# ===== RENAME KOLOM UNTUK KONSISTENSI =====

# Rename kolom agar lebih mudah dipahami dan konsisten
column_mapping = {
    'harga_oz': 'usd_per_ounce',
    'kurs_oz': 'kurs_dollar_per_ounce', 
    'idr_oz': 'idr_per_ounce',
    'harga_gr': 'usd_per_gram',
    'kurs_gr': 'kurs_dollar_per_gram',
    'idr_gr': 'idr_per_gram'
}

df = df.rename(columns=column_mapping)

print("✅ Rename kolom selesai!")
print("Mapping kolom:")
for old, new in column_mapping.items():
    print(f"  {old} → {new}")

print(f"\nKolom final: {list(df.columns)}")


✅ Rename kolom selesai!
Mapping kolom:
  harga_oz → usd_per_ounce
  kurs_oz → kurs_dollar_per_ounce
  idr_oz → idr_per_ounce
  harga_gr → usd_per_gram
  kurs_gr → kurs_dollar_per_gram
  idr_gr → idr_per_gram

Kolom final: ['tanggal', 'usd_per_ounce', 'kurs_dollar_per_ounce', 'idr_per_ounce', 'usd_per_gram', 'kurs_dollar_per_gram', 'idr_per_gram']


In [10]:
# ===== VALIDASI FINAL & SIMPAN DATA =====

print("=== HASIL AKHIR DATA CLEANING ===")
print(f"Jumlah baris: {len(df)}")
print(f"Jumlah kolom: {len(df.columns)}")
print(f"Range tanggal: {df['tanggal'].min()} hingga {df['tanggal'].max()}")

print("\n=== INFO DATASET FINAL ===")
print(df.info())

print("\n=== PREVIEW DATA FINAL ===")
print("5 data pertama:")
print(df.head())
print("\n5 data terakhir:")
print(df.tail())

# Simpan data yang sudah dibersihkan
output_file = 'harga_emas_cleaned.csv'
df.to_csv(output_file, index=False)
print(f"\n✅ Data berhasil disimpan ke '{output_file}'")


=== HASIL AKHIR DATA CLEANING ===
Jumlah baris: 1899
Jumlah kolom: 7
Range tanggal: 2020-01-01 00:00:00 hingga 2025-03-13 00:00:00

=== INFO DATASET FINAL ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1899 entries, 0 to 1898
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tanggal                1899 non-null   datetime64[ns]
 1   usd_per_ounce          1899 non-null   float64       
 2   kurs_dollar_per_ounce  1899 non-null   float64       
 3   idr_per_ounce          1899 non-null   int64         
 4   usd_per_gram           1899 non-null   float64       
 5   kurs_dollar_per_gram   1899 non-null   float64       
 6   idr_per_gram           1899 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 104.0 KB
None

=== PREVIEW DATA FINAL ===
5 data pertama:
     tanggal  usd_per_ounce  kurs_dollar_per_ounce  idr_per_ounce  \
0 2020-01-01   

## 5. SUMMARY CLEANING

**Proses cleaning yang telah dilakukan:**

1. ✅ **Load data** dari `harga_emas_febuari_2020-2025.csv`
2. ✅ **Eksplorasi awal** - cek struktur, tipe data, missing values
3. ✅ **Cleaning tanggal** - konversi dari string ke datetime
4. ✅ **Cleaning data numerik** - hapus kurung, format ribuan/desimal, konversi ke float
5. ✅ **Hapus kolom tidak perlu** - buang kolom `update_asli`
6. ✅ **Rename kolom** - untuk konsistensi dan kemudahan analisis
7. ✅ **Simpan data bersih** ke `harga_emas_cleaned.csv`


---

# Data Cleaning untuk Federal Funds Rate

Sekarang kita akan melakukan cleaning untuk dataset **Federal Funds Composite Interest Rate** dari file `Data Historis Federal Funds Composite Interest Rate.csv`

## Dataset Federal Funds Rate
Dataset ini berisi data historis tingkat suku bunga Federal Reserve dengan kolom:
- **Tanggal**: Format DD/MM/YYYY
- **Terakhir**: Nilai suku bunga penutupan
- **Pembukaan**: Nilai suku bunga pembukaan
- **Tertinggi**: Nilai tertinggi hari itu
- **Terendah**: Nilai terendah hari itu  
- **Vol.**: Volume transaksi
- **Perubahan%**: Persentase perubahan

---

In [21]:
# ===== LOAD DATA FEDERAL FUNDS RATE =====

# Load data Federal Funds Rate
df_fed = pd.read_csv('Data Historis Federal Funds Composite Interest Rate.csv')

print("=== INFORMASI DASAR DATASET FEDERAL FUNDS RATE ===")
print(f"Jumlah baris: {len(df_fed)}")
print(f"Jumlah kolom: {len(df_fed.columns)}")
print(f"Kolom: {list(df_fed.columns)}")

print("\n=== PREVIEW DATA MENTAH ===")
print("5 data pertama:")
print(df_fed.head())
print("\n5 data terakhir:")
print(df_fed.tail())

print("\n=== TIPE DATA ===")
print(df_fed.dtypes)

print("\n=== MISSING VALUES ===")
print(df_fed.isna().sum())

print("\n=== CONTOH FORMAT DATA ===")
print("Sample tanggal:", df_fed['Tanggal'].iloc[0])
print("Sample nilai suku bunga:", df_fed['Terakhir'].iloc[0])
print("Sample volume:", df_fed['Vol.'].iloc[0])
print("Sample perubahan%:", df_fed['Perubahan%'].iloc[0])

=== INFORMASI DASAR DATASET FEDERAL FUNDS RATE ===
Jumlah baris: 1302
Jumlah kolom: 7
Kolom: ['Tanggal', 'Terakhir', 'Pembukaan', 'Tertinggi', 'Terendah', 'Vol.', 'Perubahan%']

=== PREVIEW DATA MENTAH ===
5 data pertama:
      Tanggal Terakhir Pembukaan Tertinggi Terendah  Vol. Perubahan%
0  18/03/2025     4,33      4,33      4,33     4,33   NaN      0,00%
1  12/03/2025     4,33      4,33      4,33     4,33   NaN      0,00%
2  10/03/2025     4,33      4,33      4,33     4,33   NaN      0,00%
3  05/03/2025     4,33      4,33      4,33     4,33   NaN      0,00%
4  04/03/2025     4,33      4,33      4,33     4,33   NaN      0,00%

5 data terakhir:
         Tanggal Terakhir Pembukaan Tertinggi Terendah  Vol. Perubahan%
1297  07/01/2020     1,55      1,55      1,55     1,55   NaN      0,00%
1298  06/01/2020     1,55      1,55      1,55     1,55   NaN      0,00%
1299  03/01/2020     1,55      1,55      1,55     1,55   NaN      0,00%
1300  02/01/2020     1,55      1,55      1,55     1,55   N

## 1. CLEANING TANGGAL FEDERAL FUNDS RATE

Format tanggal saat ini: "DD/MM/YYYY" (contoh: "20/06/2025")
Perlu dikonversi ke format datetime untuk konsistensi dengan data emas.

In [22]:
# ===== KONVERSI TANGGAL FEDERAL FUNDS RATE =====

# Konversi tanggal dari format DD/MM/YYYY ke datetime
print("Memproses konversi tanggal Federal Funds Rate...")

# Konversi tanggal dengan format DD/MM/YYYY
df_fed['tanggal_converted'] = pd.to_datetime(df_fed['Tanggal'], format='%d/%m/%Y', errors='coerce')

# Replace kolom tanggal asli dengan yang sudah dikonversi
df_fed['Tanggal'] = df_fed['tanggal_converted']
df_fed.drop(columns=['tanggal_converted'], inplace=True)

print("✅ Konversi tanggal Federal Funds Rate selesai!")
print(f"Tanggal berhasil dikonversi: {df_fed['Tanggal'].notna().sum()}")
print(f"Tanggal gagal dikonversi: {df_fed['Tanggal'].isna().sum()}")

print(f"\nRange tanggal: {df_fed['Tanggal'].min()} hingga {df_fed['Tanggal'].max()}")

print("\n=== PREVIEW TANGGAL SETELAH KONVERSI ===")
df_fed[['Tanggal']].head(10)

Memproses konversi tanggal Federal Funds Rate...
✅ Konversi tanggal Federal Funds Rate selesai!
Tanggal berhasil dikonversi: 1302
Tanggal gagal dikonversi: 0

Range tanggal: 2019-12-31 00:00:00 hingga 2025-03-18 00:00:00

=== PREVIEW TANGGAL SETELAH KONVERSI ===


,Tanggal
0,2025-03-18
1,2025-03-12
2,2025-03-10
3,2025-03-05
4,2025-03-04
5,2025-03-03
6,2025-02-28
7,2025-02-27
8,2025-02-26
9,2025-02-25


## 2. CLEANING DATA NUMERIK FEDERAL FUNDS RATE

Kolom numerik yang perlu dibersihkan:
- **Terakhir, Pembukaan, Tertinggi, Terendah**: Menggunakan koma sebagai desimal
- **Vol.**: Berisi "B" untuk Billion, ada nilai kosong
- **Perubahan%**: Menggunakan format "%", koma sebagai desimal

In [23]:
# ===== CLEANING DATA NUMERIK FEDERAL FUNDS RATE =====

def clean_fed_numeric_column(series, is_percentage=False):
    """
    Fungsi untuk membersihkan kolom numerik Federal Funds Rate:
    - Ganti koma dengan titik untuk desimal
    - Hapus simbol % jika ada
    - Konversi ke float
    """
    cleaned = series.astype(str)
    
    # Ganti koma dengan titik untuk desimal
    cleaned = cleaned.str.replace(',', '.', regex=False)
    
    # Hapus simbol % jika kolom persentase
    if is_percentage:
        cleaned = cleaned.str.replace('%', '', regex=False)
    
    # Konversi ke float
    return pd.to_numeric(cleaned, errors='coerce')

def clean_volume_column(series):
    """
    Fungsi khusus untuk membersihkan kolom Volume:
    - Konversi "B" (Billion) ke angka
    - Handle nilai kosong
    """
    cleaned = series.astype(str)
    
    # Replace empty strings dengan NaN
    cleaned = cleaned.replace('', pd.NA)
    
    # Konversi format "XXX,00B" ke angka dalam miliar
    mask_billion = cleaned.str.contains('B', na=False)
    
    # Untuk nilai dengan "B", hapus "B" dan konversi
    cleaned.loc[mask_billion] = cleaned.loc[mask_billion].str.replace('B', '', regex=False)
    cleaned.loc[mask_billion] = cleaned.loc[mask_billion].str.replace(',', '.', regex=False)
    
    # Konversi ke float
    result = pd.to_numeric(cleaned, errors='coerce')
    
    # Kalikan dengan 1 miliar untuk nilai yang aslinya punya "B"
    result.loc[mask_billion] = result.loc[mask_billion] * 1_000_000_000
    
    return result

print("=== CLEANING KOLOM NUMERIK FEDERAL FUNDS RATE ===")

# Clean kolom suku bunga (Terakhir, Pembukaan, Tertinggi, Terendah)
interest_rate_columns = ['Terakhir', 'Pembukaan', 'Tertinggi', 'Terendah']

for col in interest_rate_columns:
    print(f"Memproses kolom: {col}")
    print(f"  Contoh sebelum: {df_fed[col].iloc[0]}")
    
    df_fed[col] = clean_fed_numeric_column(df_fed[col])
    
    print(f"  Contoh sesudah: {df_fed[col].iloc[0]}")
    print(f"  Missing values: {df_fed[col].isna().sum()}")
    print()

# Clean kolom Volume
print("Memproses kolom: Vol.")
print(f"  Contoh sebelum: {df_fed['Vol.'].iloc[2]}")  # Gunakan index 2 karena bisa ada empty

df_fed['Vol.'] = clean_volume_column(df_fed['Vol.'])

print(f"  Contoh sesudah: {df_fed['Vol.'].iloc[2]}")
print(f"  Missing values: {df_fed['Vol.'].isna().sum()}")
print()

# Clean kolom Perubahan%
print("Memproses kolom: Perubahan%")
print(f"  Contoh sebelum: {df_fed['Perubahan%'].iloc[0]}")

df_fed['Perubahan%'] = clean_fed_numeric_column(df_fed['Perubahan%'], is_percentage=True)

print(f"  Contoh sesudah: {df_fed['Perubahan%'].iloc[0]}")
print(f"  Missing values: {df_fed['Perubahan%'].isna().sum()}")

print("\n✅ Cleaning kolom numerik Federal Funds Rate selesai!")

=== CLEANING KOLOM NUMERIK FEDERAL FUNDS RATE ===
Memproses kolom: Terakhir
  Contoh sebelum: 4,33
  Contoh sesudah: 4.33
  Missing values: 0

Memproses kolom: Pembukaan
  Contoh sebelum: 4,33
  Contoh sesudah: 4.33
  Missing values: 0

Memproses kolom: Tertinggi
  Contoh sebelum: 4,33
  Contoh sesudah: 4.33
  Missing values: 0

Memproses kolom: Terendah
  Contoh sebelum: 4,33
  Contoh sesudah: 4.33
  Missing values: 0

Memproses kolom: Vol.
  Contoh sebelum: nan
  Contoh sesudah: nan
  Missing values: 1302

Memproses kolom: Perubahan%
  Contoh sebelum: 0,00%
  Contoh sesudah: 0.0
  Missing values: 0

✅ Cleaning kolom numerik Federal Funds Rate selesai!


In [24]:
# ===== VALIDASI DAN RENAME KOLOM FEDERAL FUNDS RATE =====

print("=== VALIDASI HASIL CLEANING FEDERAL FUNDS RATE ===")
print(df_fed.dtypes)

print("\n=== STATISTIK DESKRIPTIF ===")
print(df_fed.describe())

print("\n=== RENAME KOLOM UNTUK KONSISTENSI ===")
# Rename kolom agar konsisten dan mudah dipahami
fed_column_mapping = {
    'Tanggal': 'tanggal',
    'Terakhir': 'close_rate',
    'Pembukaan': 'open_rate', 
    'Tertinggi': 'high_rate',
    'Terendah': 'low_rate',
    'Vol.': 'volume_billion',
    'Perubahan%': 'change_percent'
}

df_fed = df_fed.rename(columns=fed_column_mapping)

print("Mapping kolom Federal Funds Rate:")
for old, new in fed_column_mapping.items():
    print(f"  {old} → {new}")

print(f"\nKolom final Federal Funds Rate: {list(df_fed.columns)}")

print("\n=== CONTOH DATA SETELAH CLEANING ===")
df_fed.head()

=== VALIDASI HASIL CLEANING FEDERAL FUNDS RATE ===
Tanggal       datetime64[ns]
Terakhir             float64
Pembukaan            float64
Tertinggi            float64
Terendah             float64
Vol.                 float64
Perubahan%           float64
dtype: object

=== STATISTIK DESKRIPTIF ===
                             Tanggal     Terakhir    Pembukaan    Tertinggi  \
count                           1302  1302.000000  1302.000000  1302.000000   
mean   2022-08-02 01:47:16.866359552     2.517934     2.517942     2.517942   
min              2019-12-31 00:00:00     0.040000     0.040000     0.040000   
25%              2021-04-16 18:00:00     0.090000     0.090000     0.090000   
50%              2022-08-01 12:00:00     2.330000     2.330000     2.330000   
75%              2023-11-15 18:00:00     5.080000     5.080000     5.080000   
max              2025-03-18 00:00:00     5.330000     5.330000     5.330000   
std                              NaN     2.307796     2.307805     2.3

,tanggal,close_rate,open_rate,high_rate,low_rate,volume_billion,change_percent
0,2025-03-18,4.33,4.33,4.33,4.33,NaN,0.0
1,2025-03-12,4.33,4.33,4.33,4.33,NaN,0.0
2,2025-03-10,4.33,4.33,4.33,4.33,NaN,0.0
3,2025-03-05,4.33,4.33,4.33,4.33,NaN,0.0
4,2025-03-04,4.33,4.33,4.33,4.33,NaN,0.0


In [ ]:
# ===== SIMPAN DATA FEDERAL FUNDS RATE YANG SUDAH DIBERSIHKAN =====

print("=== HASIL AKHIR FEDERAL FUNDS RATE CLEANING ===")
print(f"Jumlah baris: {len(df_fed)}")
print(f"Jumlah kolom: {len(df_fed.columns)}")
print(f"Range tanggal: {df_fed['tanggal'].min()} hingga {df_fed['tanggal'].max()}")

print("\n=== INFO DATASET FINAL FEDERAL FUNDS RATE ===")
print(df_fed.info())

print("\n=== PREVIEW DATA FINAL FEDERAL FUNDS RATE ===")
print("5 data pertama:")
print(df_fed.head())
print("\n5 data terakhir:")
print(df_fed.tail())

# Simpan data yang sudah dibersihkan dengan nama file baru
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
fed_output_file = f'federal_funds_rate_cleaned_{timestamp}.csv'
df_fed.to_csv(fed_output_file, index=False)
print(f"\n✅ Data Federal Funds Rate berhasil disimpan ke '{fed_output_file}'")

=== HASIL AKHIR FEDERAL FUNDS RATE CLEANING ===
Jumlah baris: 1302
Jumlah kolom: 7
Range tanggal: 2019-12-31 00:00:00 hingga 2025-03-18 00:00:00

=== INFO DATASET FINAL FEDERAL FUNDS RATE ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tanggal         1302 non-null   datetime64[ns]
 1   close_rate      1302 non-null   float64       
 2   open_rate       1302 non-null   float64       
 3   high_rate       1302 non-null   float64       
 4   low_rate        1302 non-null   float64       
 5   volume_billion  0 non-null      float64       
 6   change_percent  1302 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 71.3 KB
None

=== PREVIEW DATA FINAL FEDERAL FUNDS RATE ===
5 data pertama:
     tanggal  close_rate  open_rate  high_rate  low_rate  volume_billion  \
0 2025-03-18        4.33      

## 3. SUMMARY CLEANING FEDERAL FUNDS RATE

**Proses cleaning Federal Funds Rate yang telah dilakukan:**

1. ✅ **Load data** dari `Data Historis Federal Funds Composite Interest Rate.csv`
2. ✅ **Cleaning tanggal** - konversi dari DD/MM/YYYY ke datetime
3. ✅ **Cleaning suku bunga** - konversi koma ke titik desimal untuk float
4. ✅ **Cleaning volume** - handle format "XXX,XXB" dan konversi ke angka
5. ✅ **Cleaning persentase** - hapus simbol % dan konversi ke float
6. ✅ **Rename kolom** - untuk konsistensi dan kemudahan analisis
7. ✅ **Simpan data bersih** ke `federal_funds_rate_cleaned.csv`

---

# 📊 SUMMARY LENGKAP - KEDUA DATASET

**Dataset yang telah dibersihkan:**

## 🥇 Dataset Harga Emas (`harga_emas_cleaned.csv`)
- **Periode**: 2020-01-01 hingga 2025-03-13 (1,899 hari)
- **Kolom**: tanggal, usd_per_ounce, kurs_dollar_per_ounce, idr_per_ounce, usd_per_gram, kurs_dollar_per_gram, idr_per_gram

## 🏦 Dataset Federal Funds Rate (`federal_funds_rate_cleaned.csv`)
- **Periode**: 2020-01-02 hingga 2025-06-20 (~1,366 hari)
- **Kolom**: tanggal, close_rate, open_rate, high_rate, low_rate, volume_billion, change_percent

**Kedua dataset siap untuk:**
- ✅ Analisis korelasi harga emas vs suku bunga Fed
- ✅ Visualisasi time series
- ✅ Modeling prediktif
- ✅ Dashboard finansial

---

# Data Cleaning untuk Jakarta Stock Exchange Composite

Sekarang kita akan melakukan cleaning untuk dataset **Jakarta Stock Exchange Composite** dari file `Data Historis Jakarta Stock Exchange Composite.csv`

## Dataset Jakarta Stock Exchange (JCI)
Dataset ini berisi data historis indeks saham gabungan Jakarta dengan kolom:
- **Tanggal**: Format DD/MM/YYYY  
- **Terakhir**: Nilai penutupan indeks
- **Pembukaan**: Nilai pembukaan indeks
- **Tertinggi**: Nilai tertinggi hari itu
- **Terendah**: Nilai terendah hari itu
- **Vol.**: Volume transaksi (dalam format B untuk Billion)
- **Perubahan%**: Persentase perubahan

---

In [4]:
# ===== LOAD DATA JAKARTA STOCK EXCHANGE =====

# Load data Jakarta Stock Exchange Composite
df_jci = pd.read_csv('Data Historis Jakarta Stock Exchange Composite.csv')

print("=== INFORMASI DASAR DATASET JAKARTA STOCK EXCHANGE ===")
print(f"Jumlah baris: {len(df_jci)}")
print(f"Jumlah kolom: {len(df_jci.columns)}")
print(f"Kolom: {list(df_jci.columns)}")

print("\n=== PREVIEW DATA MENTAH ===")
print("5 data pertama:")
print(df_jci.head())
print("\n5 data terakhir:")
print(df_jci.tail())

print("\n=== TIPE DATA ===")
print(df_jci.dtypes)

print("\n=== MISSING VALUES ===")
print(df_jci.isna().sum())

print("\n=== CONTOH FORMAT DATA ===")
print("Sample tanggal:", df_jci['Tanggal'].iloc[0])
print("Sample nilai indeks:", df_jci['Terakhir'].iloc[0])
print("Sample volume:", df_jci['Vol.'].iloc[0])
print("Sample perubahan%:", df_jci['Perubahan%'].iloc[0])

=== INFORMASI DASAR DATASET JAKARTA STOCK EXCHANGE ===
Jumlah baris: 1262
Jumlah kolom: 7
Kolom: ['Tanggal', 'Terakhir', 'Pembukaan', 'Tertinggi', 'Terendah', 'Vol.', 'Perubahan%']

=== PREVIEW DATA MENTAH ===
5 data pertama:
      Tanggal  Terakhir Pembukaan Tertinggi  Terendah    Vol. Perubahan%
0  13/03/2025  6.647,42  6.700,11  6.707,39  6.618,54  13,99B     -0,26%
1  12/03/2025  6.665,04  6.566,80  6.665,04  6.566,20  13,97B      1,82%
2  11/03/2025  6.545,85  6.535,87  6.567,58  6.499,04  15,00B     -0,79%
3  10/03/2025  6.598,21  6.618,92  6.629,44  6.568,13  15,36B     -0,57%
4  07/03/2025  6.636,00  6.631,31  6.682,94  6.577,83  19,84B      0,27%

5 data terakhir:
         Tanggal  Terakhir Pembukaan Tertinggi  Terendah   Vol. Perubahan%
1257  07/01/2020  6.279,35  6.272,22  6.284,89  6.246,13  3,94B      0,35%
1258  06/01/2020  6.257,40  6.293,50  6.300,44  6.252,64  4,14B     -1,04%
1259  03/01/2020  6.323,47  6.306,19  6.323,47  6.287,71  4,46B      0,63%
1260  02/01/2020  

## 1. CLEANING TANGGAL JAKARTA STOCK EXCHANGE

Format tanggal saat ini: "DD/MM/YYYY" (contoh: "13/03/2025")
Perlu dikonversi ke format datetime untuk konsistensi dengan dataset lainnya.

In [5]:
# ===== KONVERSI TANGGAL JAKARTA STOCK EXCHANGE =====

# Konversi tanggal dari format DD/MM/YYYY ke datetime
print("Memproses konversi tanggal Jakarta Stock Exchange...")

# Konversi tanggal dengan format DD/MM/YYYY
df_jci['tanggal_converted'] = pd.to_datetime(df_jci['Tanggal'], format='%d/%m/%Y', errors='coerce')

# Replace kolom tanggal asli dengan yang sudah dikonversi
df_jci['Tanggal'] = df_jci['tanggal_converted']
df_jci.drop(columns=['tanggal_converted'], inplace=True)

print("✅ Konversi tanggal Jakarta Stock Exchange selesai!")
print(f"Tanggal berhasil dikonversi: {df_jci['Tanggal'].notna().sum()}")
print(f"Tanggal gagal dikonversi: {df_jci['Tanggal'].isna().sum()}")

print(f"\nRange tanggal: {df_jci['Tanggal'].min()} hingga {df_jci['Tanggal'].max()}")

print("\n=== PREVIEW TANGGAL SETELAH KONVERSI ===")
df_jci[['Tanggal']].head(10)

Memproses konversi tanggal Jakarta Stock Exchange...
✅ Konversi tanggal Jakarta Stock Exchange selesai!
Tanggal berhasil dikonversi: 1262
Tanggal gagal dikonversi: 0

Range tanggal: 2019-12-30 00:00:00 hingga 2025-03-13 00:00:00

=== PREVIEW TANGGAL SETELAH KONVERSI ===


,Tanggal
0,2025-03-13
1,2025-03-12
2,2025-03-11
3,2025-03-10
4,2025-03-07
5,2025-03-06
6,2025-03-05
7,2025-03-04
8,2025-03-03
9,2025-02-28


## 2. CLEANING DATA NUMERIK JAKARTA STOCK EXCHANGE

Kolom numerik yang perlu dibersihkan:
- **Terakhir, Pembukaan, Tertinggi, Terendah**: Menggunakan titik sebagai pemisah ribuan dan koma sebagai desimal
- **Vol.**: Berisi "B" untuk Billion, format koma desimal
- **Perubahan%**: Menggunakan format "%", koma sebagai desimal

In [6]:
# ===== CLEANING DATA NUMERIK JAKARTA STOCK EXCHANGE =====

def clean_jci_numeric_column(series, is_percentage=False):
    """
    Fungsi untuk membersihkan kolom numerik Jakarta Stock Exchange:
    - Hapus titik pemisah ribuan
    - Ganti koma dengan titik untuk desimal
    - Hapus simbol % jika ada
    - Konversi ke float
    """
    cleaned = series.astype(str)
    
    # Hapus titik pemisah ribuan KECUALI yang terakhir (desimal)
    # Contoh: "6.647,42" -> hapus titik pertama -> "6647,42"
    def remove_thousand_separator(text):
        if pd.isna(text) or text == 'nan':
            return text
        # Split berdasarkan koma untuk memisahkan bagian integer dan desimal
        parts = text.split(',')
        if len(parts) == 2:
            # Ada desimal
            integer_part = parts[0].replace('.', '')  # Hapus semua titik di bagian integer
            return f"{integer_part},{parts[1]}"
        else:
            # Tidak ada desimal, hapus semua titik
            return text.replace('.', '')
    
    cleaned = cleaned.apply(remove_thousand_separator)
    
    # Ganti koma dengan titik untuk desimal
    cleaned = cleaned.str.replace(',', '.', regex=False)
    
    # Hapus simbol % jika kolom persentase
    if is_percentage:
        cleaned = cleaned.str.replace('%', '', regex=False)
    
    # Konversi ke float
    return pd.to_numeric(cleaned, errors='coerce')

def clean_jci_volume_column(series):
    """
    Fungsi khusus untuk membersihkan kolom Volume Jakarta Stock Exchange:
    - Konversi "B" (Billion) ke angka
    - Handle format dengan koma desimal
    """
    cleaned = series.astype(str)
    
    # Replace empty strings dengan NaN
    cleaned = cleaned.replace('', pd.NA)
    
    # Konversi format "XX,XXB" ke angka dalam miliar
    mask_billion = cleaned.str.contains('B', na=False)
    
    # Untuk nilai dengan "B", hapus "B" dan konversi
    cleaned.loc[mask_billion] = cleaned.loc[mask_billion].str.replace('B', '', regex=False)
    cleaned.loc[mask_billion] = cleaned.loc[mask_billion].str.replace(',', '.', regex=False)
    
    # Konversi ke float
    result = pd.to_numeric(cleaned, errors='coerce')
    
    # Kalikan dengan 1 miliar untuk nilai yang aslinya punya "B"
    result.loc[mask_billion] = result.loc[mask_billion] * 1_000_000_000
    
    return result

print("=== CLEANING KOLOM NUMERIK JAKARTA STOCK EXCHANGE ===")

# Clean kolom indeks saham (Terakhir, Pembukaan, Tertinggi, Terendah)
stock_index_columns = ['Terakhir', 'Pembukaan', 'Tertinggi', 'Terendah']

for col in stock_index_columns:
    print(f"Memproses kolom: {col}")
    print(f"  Contoh sebelum: {df_jci[col].iloc[0]}")
    
    df_jci[col] = clean_jci_numeric_column(df_jci[col])
    
    print(f"  Contoh sesudah: {df_jci[col].iloc[0]}")
    print(f"  Missing values: {df_jci[col].isna().sum()}")
    print()

# Clean kolom Volume
print("Memproses kolom: Vol.")
print(f"  Contoh sebelum: {df_jci['Vol.'].iloc[0]}")

df_jci['Vol.'] = clean_jci_volume_column(df_jci['Vol.'])

print(f"  Contoh sesudah: {df_jci['Vol.'].iloc[0]}")
print(f"  Missing values: {df_jci['Vol.'].isna().sum()}")
print()

# Clean kolom Perubahan%
print("Memproses kolom: Perubahan%")
print(f"  Contoh sebelum: {df_jci['Perubahan%'].iloc[0]}")

df_jci['Perubahan%'] = clean_jci_numeric_column(df_jci['Perubahan%'], is_percentage=True)

print(f"  Contoh sesudah: {df_jci['Perubahan%'].iloc[0]}")
print(f"  Missing values: {df_jci['Perubahan%'].isna().sum()}")

print("\n✅ Cleaning kolom numerik Jakarta Stock Exchange selesai!")

=== CLEANING KOLOM NUMERIK JAKARTA STOCK EXCHANGE ===
Memproses kolom: Terakhir
  Contoh sebelum: 6.647,42
  Contoh sesudah: 6647.42
  Missing values: 0

Memproses kolom: Pembukaan
  Contoh sebelum: 6.700,11
  Contoh sesudah: 6700.11
  Missing values: 0

Memproses kolom: Tertinggi
  Contoh sebelum: 6.707,39
  Contoh sesudah: 6707.39
  Missing values: 0

Memproses kolom: Terendah
  Contoh sebelum: 6.618,54
  Contoh sesudah: 6618.54
  Missing values: 0

Memproses kolom: Vol.
  Contoh sebelum: 13,99B
  Contoh sesudah: 13990000000.0
  Missing values: 3

Memproses kolom: Perubahan%
  Contoh sebelum: -0,26%
  Contoh sesudah: -0.26
  Missing values: 0

✅ Cleaning kolom numerik Jakarta Stock Exchange selesai!


In [7]:
# ===== VALIDASI DAN RENAME KOLOM JAKARTA STOCK EXCHANGE =====

print("=== VALIDASI HASIL CLEANING JAKARTA STOCK EXCHANGE ===")
print(df_jci.dtypes)

print("\n=== STATISTIK DESKRIPTIF ===")
print(df_jci.describe())

print("\n=== RENAME KOLOM UNTUK KONSISTENSI ===")
# Rename kolom agar konsisten dan mudah dipahami
jci_column_mapping = {
    'Tanggal': 'tanggal',
    'Terakhir': 'close_index',
    'Pembukaan': 'open_index', 
    'Tertinggi': 'high_index',
    'Terendah': 'low_index',
    'Vol.': 'volume_billion',
    'Perubahan%': 'change_percent'
}

df_jci = df_jci.rename(columns=jci_column_mapping)

print("Mapping kolom Jakarta Stock Exchange:")
for old, new in jci_column_mapping.items():
    print(f"  {old} → {new}")

print(f"\nKolom final Jakarta Stock Exchange: {list(df_jci.columns)}")

print("\n=== CONTOH DATA SETELAH CLEANING ===")
df_jci.head()

=== VALIDASI HASIL CLEANING JAKARTA STOCK EXCHANGE ===
Tanggal       datetime64[ns]
Terakhir             float64
Pembukaan            float64
Tertinggi            float64
Terendah             float64
Vol.                 float64
Perubahan%           float64
dtype: object

=== STATISTIK DESKRIPTIF ===
                             Tanggal     Terakhir    Pembukaan    Tertinggi  \
count                           1262  1262.000000  1262.000000  1262.000000   
mean   2022-08-03 21:51:03.708399360  6531.032710  6534.446133  6570.646894   
min              2019-12-30 00:00:00  3937.630000  3937.630000  4123.560000   
25%              2021-04-19 06:00:00  6096.667500  6106.717500  6134.997500   
50%              2022-08-04 12:00:00  6788.445000  6787.080000  6819.455000   
75%              2023-11-16 18:00:00  7078.327500  7080.427500  7115.442500   
max              2025-03-13 00:00:00  7905.390000  7904.400000  7910.560000   
std                              NaN   775.379932   775.494033   7

,tanggal,close_index,open_index,high_index,low_index,volume_billion,change_percent
0,2025-03-13,6647.42,6700.11,6707.39,6618.54,1.399000e+10,-0.26
1,2025-03-12,6665.04,6566.80,6665.04,6566.20,1.397000e+10,1.82
2,2025-03-11,6545.85,6535.87,6567.58,6499.04,1.500000e+10,-0.79
3,2025-03-10,6598.21,6618.92,6629.44,6568.13,1.536000e+10,-0.57
4,2025-03-07,6636.00,6631.31,6682.94,6577.83,1.984000e+10,0.27


In [8]:
# ===== SIMPAN DATA JAKARTA STOCK EXCHANGE YANG SUDAH DIBERSIHKAN =====

print("=== HASIL AKHIR JAKARTA STOCK EXCHANGE CLEANING ===")
print(f"Jumlah baris: {len(df_jci)}")
print(f"Jumlah kolom: {len(df_jci.columns)}")
print(f"Range tanggal: {df_jci['tanggal'].min()} hingga {df_jci['tanggal'].max()}")

print("\n=== INFO DATASET FINAL JAKARTA STOCK EXCHANGE ===")
print(df_jci.info())

print("\n=== PREVIEW DATA FINAL JAKARTA STOCK EXCHANGE ===")
print("5 data pertama:")
print(df_jci.head())
print("\n5 data terakhir:")
print(df_jci.tail())

# Simpan data yang sudah dibersihkan
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
jci_output_file = f'jakarta_stock_exchange_cleaned_{timestamp}.csv'
df_jci.to_csv(jci_output_file, index=False)
print(f"\n✅ Data Jakarta Stock Exchange berhasil disimpan ke '{jci_output_file}'")

=== HASIL AKHIR JAKARTA STOCK EXCHANGE CLEANING ===
Jumlah baris: 1262
Jumlah kolom: 7
Range tanggal: 2019-12-30 00:00:00 hingga 2025-03-13 00:00:00

=== INFO DATASET FINAL JAKARTA STOCK EXCHANGE ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262 entries, 0 to 1261
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tanggal         1262 non-null   datetime64[ns]
 1   close_index     1262 non-null   float64       
 2   open_index      1262 non-null   float64       
 3   high_index      1262 non-null   float64       
 4   low_index       1262 non-null   float64       
 5   volume_billion  1259 non-null   float64       
 6   change_percent  1262 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 69.1 KB
None

=== PREVIEW DATA FINAL JAKARTA STOCK EXCHANGE ===
5 data pertama:
     tanggal  close_index  open_index  high_index  low_index  volume_billion  \
0 2025-03-13  

## 3. SUMMARY CLEANING JAKARTA STOCK EXCHANGE

**Proses cleaning Jakarta Stock Exchange yang telah dilakukan:**

1. ✅ **Load data** dari `Data Historis Jakarta Stock Exchange Composite.csv`
2. ✅ **Cleaning tanggal** - konversi dari DD/MM/YYYY ke datetime
3. ✅ **Cleaning indeks saham** - handle format titik ribuan dan koma desimal
4. ✅ **Cleaning volume** - handle format "XX,XXB" dan konversi ke angka
5. ✅ **Cleaning persentase** - hapus simbol % dan konversi ke float
6. ✅ **Rename kolom** - untuk konsistensi dan kemudahan analisis
7. ✅ **Simpan data bersih** ke `jakarta_stock_exchange_cleaned_TIMESTAMP.csv`

---

# 📊 SUMMARY LENGKAP - KETIGA DATASET

**Dataset yang telah dibersihkan:**

## 🥇 Dataset Harga Emas (`harga_emas_cleaned.csv`)
- **Periode**: 2020-01-01 hingga 2025-03-13 (1,899 hari)
- **Kolom**: tanggal, usd_per_ounce, kurs_dollar_per_ounce, idr_per_ounce, usd_per_gram, kurs_dollar_per_gram, idr_per_gram

## 🏦 Dataset Federal Funds Rate (`federal_funds_rate_cleaned_TIMESTAMP.csv`)
- **Periode**: 2019-12-31 hingga 2025-03-18 (~1,302 hari)
- **Kolom**: tanggal, close_rate, open_rate, high_rate, low_rate, volume_billion, change_percent

## 📈 Dataset Jakarta Stock Exchange (`jakarta_stock_exchange_cleaned_TIMESTAMP.csv`)
- **Periode**: 2019-12-30 hingga 2025-03-13 (~1,262 hari)
- **Kolom**: tanggal, close_index, open_index, high_index, low_index, volume_billion, change_percent

---

## 🎯 **ANALISIS YANG DAPAT DILAKUKAN:**

### **📊 Multi-Asset Analysis**
- ✅ **Korelasi harga emas vs suku bunga Fed vs indeks saham**
- ✅ **Time series analysis untuk ketiga aset**
- ✅ **Volatility comparison**
- ✅ **Risk-return analysis**

### **🔮 Predictive Modeling**
- ✅ **Multi-variate forecasting**
- ✅ **Cross-asset impact modeling**
- ✅ **Economic indicator integration**

### **📈 Financial Dashboard**
- ✅ **Real-time portfolio tracking**
- ✅ **Economic sentiment analysis**
- ✅ **Investment decision support**

**Semua dataset telah dibersihkan dan siap untuk analisis finansial yang komprehensif!** 🚀

In [9]:
# === SUMMARY FINAL HASIL CLEANING SEMUA DATASET ===

print("🎉 PROSES CLEANING DATA SELESAI! 🎉")
print("="*60)

# Summary file yang sudah di-cleaning
cleaned_files = []

import os
import pandas as pd

# Cek file yang ada
base_path = r"e:\Ipan\Big data\FP"

# 1. Harga Emas
gold_file = os.path.join(base_path, "harga_emas_cleaned.csv")
if os.path.exists(gold_file):
    df_gold_summary = pd.read_csv(gold_file)
    cleaned_files.append({
        'dataset': 'Harga Emas',
        'file': 'harga_emas_cleaned.csv',
        'rows': len(df_gold_summary),
        'columns': list(df_gold_summary.columns),
        'date_range': f"{df_gold_summary['tanggal'].min()} - {df_gold_summary['tanggal'].max()}"
    })

# 2. Federal Funds Rate
import glob
ffr_files = glob.glob(os.path.join(base_path, "federal_funds_rate_cleaned_*.csv"))
if ffr_files:
    latest_ffr = max(ffr_files, key=os.path.getctime)
    df_ffr_summary = pd.read_csv(latest_ffr)
    cleaned_files.append({
        'dataset': 'Federal Funds Rate',
        'file': os.path.basename(latest_ffr),
        'rows': len(df_ffr_summary),
        'columns': list(df_ffr_summary.columns),
        'date_range': f"{df_ffr_summary['tanggal'].min()} - {df_ffr_summary['tanggal'].max()}"
    })

# 3. Jakarta Stock Exchange
jci_files = glob.glob(os.path.join(base_path, "jakarta_stock_exchange_cleaned_*.csv"))
if jci_files:
    latest_jci = max(jci_files, key=os.path.getctime)
    df_jci_summary = pd.read_csv(latest_jci)
    cleaned_files.append({
        'dataset': 'Jakarta Stock Exchange',
        'file': os.path.basename(latest_jci),
        'rows': len(df_jci_summary),
        'columns': list(df_jci_summary.columns),
        'date_range': f"{df_jci_summary['tanggal'].min()} - {df_jci_summary['tanggal'].max()}"
    })

# Tampilkan summary
for i, file_info in enumerate(cleaned_files, 1):
    print(f"\n{i}. 📊 {file_info['dataset']}")
    print(f"   📁 File: {file_info['file']}")
    print(f"   📏 Dimensi: {file_info['rows']} baris × {len(file_info['columns'])} kolom")
    print(f"   📅 Rentang tanggal: {file_info['date_range']}")
    print(f"   🏗️ Kolom: {', '.join(file_info['columns'])}")

print("\n" + "="*60)
print("✅ SEMUA DATASET SIAP UNTUK ANALISIS!")
print("🔍 Data sudah dibersihkan dan siap untuk:")
print("   • Analisis korelasi")
print("   • Visualisasi trend")
print("   • Pemodelan prediktif")
print("   • Dashboard interaktif")
print("="*60)

🎉 PROSES CLEANING DATA SELESAI! 🎉

1. 📊 Harga Emas
   📁 File: harga_emas_cleaned.csv
   📏 Dimensi: 1899 baris × 7 kolom
   📅 Rentang tanggal: 2020-01-01 - 2025-03-13
   🏗️ Kolom: tanggal, usd_per_ounce, kurs_dollar_per_ounce, idr_per_ounce, usd_per_gram, kurs_dollar_per_gram, idr_per_gram

2. 📊 Federal Funds Rate
   📁 File: federal_funds_rate_cleaned_20250630_195749.csv
   📏 Dimensi: 1302 baris × 7 kolom
   📅 Rentang tanggal: 2019-12-31 - 2025-03-18
   🏗️ Kolom: tanggal, close_rate, open_rate, high_rate, low_rate, volume_billion, change_percent

3. 📊 Jakarta Stock Exchange
   📁 File: jakarta_stock_exchange_cleaned_20250702_095758.csv
   📏 Dimensi: 1262 baris × 7 kolom
   📅 Rentang tanggal: 2019-12-30 - 2025-03-13
   🏗️ Kolom: tanggal, close_index, open_index, high_index, low_index, volume_billion, change_percent

✅ SEMUA DATASET SIAP UNTUK ANALISIS!
🔍 Data sudah dibersihkan dan siap untuk:
   • Analisis korelasi
   • Visualisasi trend
   • Pemodelan prediktif
   • Dashboard interaktif


---

# Data Cleaning untuk Data Inflasi

Sekarang kita akan melakukan cleaning untuk dataset **Data Inflasi** dari file `data inflasi.csv`

## Dataset Data Inflasi
Dataset ini berisi data historis tingkat inflasi Indonesia dalam format CSV yang lebih mudah diproses dibanding format Excel sebelumnya.

**Langkah-langkah cleaning data inflasi:**
1. **Load dan eksplorasi data CSV**
2. **Cleaning format tanggal/periode**
3. **Cleaning nilai inflasi (format persen, desimal)**
4. **Standardisasi nama kolom**
5. **Validasi dan export ke CSV final**

---

In [23]:
# ===== LOAD DAN EKSPLORASI DATA INFLASI CSV =====

# Load data inflasi dari file CSV baru
df_inflasi = pd.read_csv('data inflasi.csv')

print("=== INFORMASI DASAR DATASET DATA INFLASI ===")
print(f"Jumlah baris: {len(df_inflasi)}")
print(f"Jumlah kolom: {len(df_inflasi.columns)}")
print(f"Kolom: {list(df_inflasi.columns)}")

print("\n=== PREVIEW DATA MENTAH ===")
print("5 data pertama:")
print(df_inflasi.head())
print("\n5 data terakhir:")
print(df_inflasi.tail())

print("\n=== TIPE DATA ===")
print(df_inflasi.dtypes)

print("\n=== MISSING VALUES ===")
print(df_inflasi.isna().sum())

print("\n=== CONTOH FORMAT DATA ===")
for col in df_inflasi.columns:
    sample_vals = df_inflasi[col].dropna().head(3).tolist()
    print(f"Sample {col}: {sample_vals}")

print("\n=== IDENTIFIKASI JENIS KOLOM ===")
for col in df_inflasi.columns:
    col_lower = str(col).lower()
    if any(keyword in col_lower for keyword in ['tanggal', 'date', 'tahun', 'bulan', 'periode', 'waktu']):
        print(f"  {col} -> KANDIDAT KOLOM TANGGAL")
    elif any(keyword in col_lower for keyword in ['inflasi', 'inflation', 'persen', '%', 'rate']):
        print(f"  {col} -> KANDIDAT KOLOM INFLASI")
    else:
        print(f"  {col} -> KOLOM LAINNYA")

=== INFORMASI DASAR DATASET DATA INFLASI ===
Jumlah baris: 63
Jumlah kolom: 3
Kolom: ['No', 'Periode', 'Data Inflasi']

=== PREVIEW DATA MENTAH ===
5 data pertama:
   No        Periode Data Inflasi
0   1     Maret 2025       1.03 %
1   2  Februari 2025      -0.09 %
2   3   Januari 2025       0.76 %
3   4  Desember 2024       1.57 %
4   5  November 2024       1.55 %

5 data terakhir:
    No        Periode Data Inflasi
58  59       Mei 2020       2.19 %
59  60     April 2020       2.67 %
60  61     Maret 2020       2.96 %
61  62  Februari 2020       2.98 %
62  63   Januari 2020       2.68 %

=== TIPE DATA ===
No               int64
Periode         object
Data Inflasi    object
dtype: object

=== MISSING VALUES ===
No              0
Periode         0
Data Inflasi    0
dtype: int64

=== CONTOH FORMAT DATA ===
Sample No: [1, 2, 3]
Sample Periode: ['Maret 2025', 'Februari 2025', 'Januari 2025']
Sample Data Inflasi: ['1.03 %', '-0.09 %', '0.76 %']

=== IDENTIFIKASI JENIS KOLOM ===
  No -> KOL

## 1. CLEANING TANGGAL DATA INFLASI

Berdasarkan eksplorasi data, kita akan membersihkan kolom tanggal/periode agar konsisten dengan format datetime untuk analisis.

In [24]:
# ===== CLEANING TANGGAL DATA INFLASI =====

# Buat copy untuk cleaning
df_inflasi_clean = df_inflasi.copy()

print("=== CLEANING KOLOM TANGGAL/PERIODE ===")

# Identifikasi kolom tanggal berdasarkan nama dan isi
date_column = None
for col in df_inflasi_clean.columns:
    col_lower = str(col).lower()
    if any(keyword in col_lower for keyword in ['tanggal', 'date', 'tahun', 'bulan', 'periode', 'waktu']):
        date_column = col
        break

if date_column:
    print(f"Kolom tanggal yang ditemukan: {date_column}")
    print(f"Sample data sebelum cleaning:")
    print(df_inflasi_clean[date_column].head())
    
    # Fungsi untuk konversi berbagai format tanggal
    def clean_date_column(date_series):
        cleaned_dates = []
        
        for date_val in date_series:
            if pd.isna(date_val):
                cleaned_dates.append(pd.NaT)
                continue
            
            date_str = str(date_val).strip()
            
            try:
                # Coba berbagai format tanggal
                # Format 1: DD/MM/YYYY atau DD-MM-YYYY
                if '/' in date_str or '-' in date_str:
                    separator = '/' if '/' in date_str else '-'
                    parts = date_str.split(separator)
                    if len(parts) == 3:
                        # Coba DD/MM/YYYY
                        try:
                            cleaned_date = pd.to_datetime(date_str, format=f'%d{separator}%m{separator}%Y')
                            cleaned_dates.append(cleaned_date)
                            continue
                        except:
                            pass
                        
                        # Coba MM/DD/YYYY  
                        try:
                            cleaned_date = pd.to_datetime(date_str, format=f'%m{separator}%d{separator}%Y')
                            cleaned_dates.append(cleaned_date)
                            continue
                        except:
                            pass
                
                # Format 2: YYYY-MM-DD
                try:
                    cleaned_date = pd.to_datetime(date_str, format='%Y-%m-%d')
                    cleaned_dates.append(cleaned_date)
                    continue
                except:
                    pass
                
                # Format 3: "Bulan Tahun" seperti "Januari 2020"
                month_mapping = {
                    'januari': 1, 'februari': 2, 'maret': 3, 'april': 4,
                    'mei': 5, 'juni': 6, 'juli': 7, 'agustus': 8,
                    'september': 9, 'oktober': 10, 'november': 11, 'desember': 12,
                    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
                }
                
                parts = date_str.lower().split()
                if len(parts) >= 2:
                    month_name = parts[0]
                    year_str = parts[1]
                    
                    if month_name in month_mapping and year_str.isdigit():
                        month_num = month_mapping[month_name]
                        year_num = int(year_str)
                        cleaned_date = pd.to_datetime(f"{year_num}-{month_num:02d}-01")
                        cleaned_dates.append(cleaned_date)
                        continue
                
                # Jika semua format gagal, coba pandas to_datetime otomatis
                try:
                    cleaned_date = pd.to_datetime(date_str)
                    cleaned_dates.append(cleaned_date)
                    continue
                except:
                    pass
                
                # Jika masih gagal, set sebagai NaT
                cleaned_dates.append(pd.NaT)
                
            except Exception as e:
                cleaned_dates.append(pd.NaT)
        
        return pd.Series(cleaned_dates)
    
    # Apply cleaning function
    df_inflasi_clean['tanggal'] = clean_date_column(df_inflasi_clean[date_column])
    
    print(f"\n✅ Konversi tanggal selesai!")
    print(f"Tanggal berhasil dikonversi: {df_inflasi_clean['tanggal'].notna().sum()}")
    print(f"Tanggal gagal dikonversi: {df_inflasi_clean['tanggal'].isna().sum()}")
    
    if df_inflasi_clean['tanggal'].notna().sum() > 0:
        print(f"Range tanggal: {df_inflasi_clean['tanggal'].min()} hingga {df_inflasi_clean['tanggal'].max()}")
    
    print(f"\nSample tanggal setelah cleaning:")
    print(df_inflasi_clean['tanggal'].head())
    
else:
    print("❌ Tidak ditemukan kolom tanggal yang jelas")
    print("💡 Silakan tentukan kolom mana yang berisi informasi tanggal")

=== CLEANING KOLOM TANGGAL/PERIODE ===
Kolom tanggal yang ditemukan: Periode
Sample data sebelum cleaning:
0       Maret 2025
1    Februari 2025
2     Januari 2025
3    Desember 2024
4    November 2024
Name: Periode, dtype: object

✅ Konversi tanggal selesai!
Tanggal berhasil dikonversi: 63
Tanggal gagal dikonversi: 0
Range tanggal: 2020-01-01 00:00:00 hingga 2025-03-01 00:00:00

Sample tanggal setelah cleaning:
0   2025-03-01
1   2025-02-01
2   2025-01-01
3   2024-12-01
4   2024-11-01
Name: tanggal, dtype: datetime64[ns]


## 2. CLEANING NILAI INFLASI

Membersihkan kolom yang berisi nilai inflasi dari format string dengan simbol % ke format numerik.

In [25]:
# ===== CLEANING NILAI INFLASI =====

print("=== CLEANING KOLOM NILAI INFLASI ===")

# Identifikasi kolom inflasi
inflation_column = None
for col in df_inflasi_clean.columns:
    col_lower = str(col).lower()
    if any(keyword in col_lower for keyword in ['inflasi', 'inflation', 'persen', '%', 'rate']) and col != 'tanggal':
        inflation_column = col
        break

# Jika tidak ditemukan berdasarkan nama, cari kolom numerik yang bukan tanggal
if not inflation_column:
    for col in df_inflasi_clean.columns:
        if col not in ['tanggal', date_column] and col != date_column:
            # Cek apakah kolom berisi nilai yang bisa jadi inflasi
            sample_vals = df_inflasi_clean[col].dropna().head(5).astype(str)
            if any('%' in str(val) or any(char.isdigit() for char in str(val)) for val in sample_vals):
                inflation_column = col
                break

if inflation_column:
    print(f"Kolom inflasi yang ditemukan: {inflation_column}")
    print(f"Sample data sebelum cleaning:")
    print(df_inflasi_clean[inflation_column].head())
    
    def clean_inflation_value(val):
        """
        Fungsi untuk membersihkan nilai inflasi:
        - Hapus simbol %
        - Ganti koma dengan titik untuk desimal
        - Konversi ke float
        """
        if pd.isna(val):
            return val
        
        # Konversi ke string untuk processing
        val_str = str(val).strip()
        
        # Hapus simbol %
        val_str = val_str.replace('%', '')
        
        # Ganti koma dengan titik untuk desimal (format Indonesia)
        val_str = val_str.replace(',', '.')
        
        # Hapus spasi tambahan
        val_str = val_str.strip()
        
        try:
            # Konversi ke float
            return float(val_str)
        except:
            # Jika gagal, return NaN
            return pd.NA
    
    # Apply cleaning function
    print(f"\nMemproses kolom: {inflation_column}")
    df_inflasi_clean['inflation_rate'] = df_inflasi_clean[inflation_column].apply(clean_inflation_value)
    
    print(f"✅ Cleaning nilai inflasi selesai!")
    print(f"Nilai berhasil dikonversi: {df_inflasi_clean['inflation_rate'].notna().sum()}")
    print(f"Nilai gagal dikonversi: {df_inflasi_clean['inflation_rate'].isna().sum()}")
    
    if df_inflasi_clean['inflation_rate'].notna().sum() > 0:
        print(f"Range nilai inflasi: {df_inflasi_clean['inflation_rate'].min():.2f}% hingga {df_inflasi_clean['inflation_rate'].max():.2f}%")
    
    print(f"\nSample nilai inflasi setelah cleaning:")
    print(df_inflasi_clean['inflation_rate'].head())
    
    print(f"\nPerbandingan sebelum dan sesudah:")
    comparison_df = pd.DataFrame({
        'Sebelum': df_inflasi_clean[inflation_column].head(),
        'Sesudah': df_inflasi_clean['inflation_rate'].head()
    })
    print(comparison_df)
    
else:
    print("❌ Tidak ditemukan kolom inflasi yang jelas")
    print("💡 Silakan tentukan kolom mana yang berisi nilai inflasi")
    print("Kolom yang tersedia:")
    for col in df_inflasi_clean.columns:
        print(f"  - {col}")

print(f"\n=== STATUS CLEANING SEMENTARA ===")
print(f"Kolom yang sudah dibersihkan:")
if 'tanggal' in df_inflasi_clean.columns:
    print(f"  ✅ tanggal: {df_inflasi_clean['tanggal'].notna().sum()} data valid")
if 'inflation_rate' in df_inflasi_clean.columns:
    print(f"  ✅ inflation_rate: {df_inflasi_clean['inflation_rate'].notna().sum()} data valid")

=== CLEANING KOLOM NILAI INFLASI ===
Kolom inflasi yang ditemukan: Data Inflasi
Sample data sebelum cleaning:
0     1.03 %
1    -0.09 %
2     0.76 %
3     1.57 %
4     1.55 %
Name: Data Inflasi, dtype: object

Memproses kolom: Data Inflasi
✅ Cleaning nilai inflasi selesai!
Nilai berhasil dikonversi: 63
Nilai gagal dikonversi: 0
Range nilai inflasi: -0.09% hingga 5.95%

Sample nilai inflasi setelah cleaning:
0    1.03
1   -0.09
2    0.76
3    1.57
4    1.55
Name: inflation_rate, dtype: float64

Perbandingan sebelum dan sesudah:
   Sebelum  Sesudah
0   1.03 %     1.03
1  -0.09 %    -0.09
2   0.76 %     0.76
3   1.57 %     1.57
4   1.55 %     1.55

=== STATUS CLEANING SEMENTARA ===
Kolom yang sudah dibersihkan:
  ✅ tanggal: 63 data valid
  ✅ inflation_rate: 63 data valid


## 3. FINALISASI DAN VALIDASI DATA INFLASI

Tahap akhir cleaning: seleksi kolom final, validasi data, dan export ke file CSV.

In [26]:
# ===== FINALISASI DAN SIMPAN DATA INFLASI =====

print("=== FINALISASI DATA INFLASI ===")

# Pilih kolom yang diperlukan untuk konsistensi dengan dataset lain
if 'tanggal' in df_inflasi_clean.columns and 'inflation_rate' in df_inflasi_clean.columns:
    
    # Buat DataFrame final dengan kolom yang konsisten
    df_inflasi_final = df_inflasi_clean[['tanggal', 'inflation_rate']].copy()
    
    # Hapus baris dengan missing values di kolom penting
    df_inflasi_final = df_inflasi_final.dropna(subset=['tanggal', 'inflation_rate'])
    
    # Sort berdasarkan tanggal (dari yang terlama)
    df_inflasi_final = df_inflasi_final.sort_values('tanggal').reset_index(drop=True)
    
    print(f"✅ Data inflasi berhasil dibersihkan!")
    print(f"Dimensi final: {df_inflasi_final.shape}")
    print(f"Kolom final: {list(df_inflasi_final.columns)}")
    print(f"Range tanggal: {df_inflasi_final['tanggal'].min()} hingga {df_inflasi_final['tanggal'].max()}")
    
    print("\n=== STATISTIK DATA INFLASI ===")
    stats = df_inflasi_final['inflation_rate'].describe()
    print(f"Count: {stats['count']}")
    print(f"Mean: {stats['mean']:.2f}%")
    print(f"Std: {stats['std']:.2f}%") 
    print(f"Min: {stats['min']:.2f}%")
    print(f"Max: {stats['max']:.2f}%")
    
    print("\n=== DETEKSI OUTLIER ===")
    Q1 = df_inflasi_final['inflation_rate'].quantile(0.25)
    Q3 = df_inflasi_final['inflation_rate'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df_inflasi_final[(df_inflasi_final['inflation_rate'] < lower_bound) | 
                               (df_inflasi_final['inflation_rate'] > upper_bound)]
    
    print(f"Outliers detected: {len(outliers)} data points")
    if len(outliers) > 0 and len(outliers) <= 5:
        print("Outlier values:")
        for idx, row in outliers.iterrows():
            print(f"  {row['tanggal'].strftime('%Y-%m')}: {row['inflation_rate']:.2f}%")
    
    print("\n=== PREVIEW DATA FINAL ===")
    print("5 data pertama:")
    print(df_inflasi_final.head())
    print("\n5 data terakhir:")
    print(df_inflasi_final.tail())
    
    print("\n=== VALIDASI KONSISTENSI ===")
    # Cek apakah ada gap dalam data bulanan
    date_diff = df_inflasi_final['tanggal'].diff().dt.days
    expected_diff = 30  # Sekitar 30 hari untuk data bulanan
    gaps = date_diff[(date_diff > 35) | (date_diff < 25)]  # Toleransi ±5 hari
    
    if len(gaps) > 0:
        print(f"⚠️  Ditemukan {len(gaps)} gap dalam data bulanan")
    else:
        print("✅ Data bulanan konsisten tanpa gap signifikan")
    
    # Simpan ke file CSV
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    inflasi_output_file = f'data_inflasi_cleaned_{timestamp}.csv'
    df_inflasi_final.to_csv(inflasi_output_file, index=False)
    print(f"\n✅ Data inflasi berhasil disimpan ke '{inflasi_output_file}'")
    
    print("\n=== INFO DATASET FINAL ===")
    print(df_inflasi_final.info())
    
else:
    print("❌ Kolom tanggal atau inflation_rate tidak tersedia")
    print("Data yang tersedia:")
    print(df_inflasi_clean.columns.tolist())

print("\n" + "="*60)
print("📊 STATUS CLEANING DATA INFLASI CSV")
if 'df_inflasi_final' in locals():
    print("✅ BERHASIL - Data inflasi CSV siap untuk analisis")
    print(f"📁 File output: {inflasi_output_file}")
    print(f"📏 Dimensi: {df_inflasi_final.shape}")
    print(f"📅 Periode: {df_inflasi_final['tanggal'].min().strftime('%Y-%m')} - {df_inflasi_final['tanggal'].max().strftime('%Y-%m')}")
else:
    print("❌ PERLU PERBAIKAN - Cek data atau proses cleaning")

=== FINALISASI DATA INFLASI ===
✅ Data inflasi berhasil dibersihkan!
Dimensi final: (63, 2)
Kolom final: ['tanggal', 'inflation_rate']
Range tanggal: 2020-01-01 00:00:00 hingga 2025-03-01 00:00:00

=== STATISTIK DATA INFLASI ===
Count: 63.0
Mean: 2.65%
Std: 1.38%
Min: -0.09%
Max: 5.95%

=== DETEKSI OUTLIER ===
Outliers detected: 2 data points
Outlier values:
  2022-09: 5.95%
  2022-10: 5.71%

=== PREVIEW DATA FINAL ===
5 data pertama:
     tanggal  inflation_rate
0 2020-01-01            2.68
1 2020-02-01            2.98
2 2020-03-01            2.96
3 2020-04-01            2.67
4 2020-05-01            2.19

5 data terakhir:
      tanggal  inflation_rate
58 2024-11-01            1.55
59 2024-12-01            1.57
60 2025-01-01            0.76
61 2025-02-01           -0.09
62 2025-03-01            1.03

=== VALIDASI KONSISTENSI ===
✅ Data bulanan konsisten tanpa gap signifikan

✅ Data inflasi berhasil disimpan ke 'data_inflasi_cleaned_20250702_102841.csv'

=== INFO DATASET FINAL ===
<clas

## 4. SUMMARY CLEANING DATA INFLASI

**Proses cleaning Data Inflasi CSV yang telah dilakukan:**

1. ✅ **Load data CSV** - membaca file `data inflasi.csv` yang lebih mudah diproses
2. ✅ **Identifikasi kolom** - deteksi otomatis kolom tanggal dan inflasi
3. ✅ **Cleaning tanggal** - parsing format "Bulan Tahun" menjadi datetime
4. ✅ **Cleaning nilai inflasi** - konversi dari "1.03 %" ke format float 1.03
5. ✅ **Validasi data** - cek outlier, gap, dan konsistensi temporal
6. ✅ **Standardisasi kolom** - rename ke `tanggal` dan `inflation_rate`
7. ✅ **Simpan data bersih** ke `data_inflasi_cleaned_TIMESTAMP.csv`

---

# 📊 SUMMARY LENGKAP - KEEMPAT DATASET (UPDATED)

**Dataset yang telah dibersihkan:**

## 🥇 Dataset Harga Emas (`harga_emas_cleaned.csv`)
- **Periode**: 2020-01-01 hingga 2025-03-13 (1,899 hari)
- **Kolom**: tanggal, usd_per_ounce, kurs_dollar_per_ounce, idr_per_ounce, usd_per_gram, kurs_dollar_per_gram, idr_per_gram

## 🏦 Dataset Federal Funds Rate (`federal_funds_rate_cleaned_TIMESTAMP.csv`)
- **Periode**: 2019-12-31 hingga 2025-03-18 (~1,302 hari)
- **Kolom**: tanggal, close_rate, open_rate, high_rate, low_rate, volume_billion, change_percent

## 📈 Dataset Jakarta Stock Exchange (`jakarta_stock_exchange_cleaned_TIMESTAMP.csv`)
- **Periode**: 2019-12-30 hingga 2025-03-13 (~1,262 hari)  
- **Kolom**: tanggal, close_index, open_index, high_index, low_index, volume_billion, change_percent

## 📊 Dataset Data Inflasi (`data_inflasi_cleaned_TIMESTAMP.csv`) ← **UPDATED!**
- **Periode**: 2020-01-01 hingga 2025-03-01 (63 bulan)
- **Kolom**: tanggal, inflation_rate
- **Range Inflasi**: -0.09% hingga 5.95%

---

## 🎯 **ANALISIS YANG DAPAT DILAKUKAN:**

### **📊 Multi-Asset Economic Analysis**
- ✅ **Korelasi harga emas vs suku bunga Fed vs indeks saham vs inflasi**
- ✅ **Time series analysis untuk keempat indikator ekonomi**
- ✅ **Economic indicator dashboard**
- ✅ **Inflation hedging analysis (emas vs inflasi)**

### **🔮 Advanced Economic Modeling**
- ✅ **Multi-variate forecasting dengan 4 variabel**
- ✅ **Economic cycle prediction**
- ✅ **Risk assessment model**
- ✅ **Portfolio optimization berdasarkan kondisi ekonomi**

### **📈 Comprehensive Financial Dashboard**
- ✅ **Real-time economic monitoring**
- ✅ **Inflation-adjusted returns**
- ✅ **Economic sentiment analysis**
- ✅ **Investment decision support system**

**Semua dataset telah dibersihkan dan siap untuk analisis ekonomi yang komprehensif!** 🚀

In [27]:
# === SUMMARY FINAL SEMUA DATASET TERMASUK DATA INFLASI ===

print("🎉 PROSES CLEANING SEMUA DATA SELESAI! 🎉")
print("="*60)

# Summary semua file yang sudah di-cleaning
all_cleaned_files = []

import os
import pandas as pd

# Cek file yang ada
base_path = r"e:\Ipan\Big data\FP"

# 1. Harga Emas
gold_file = os.path.join(base_path, "harga_emas_cleaned.csv")
if os.path.exists(gold_file):
    df_summary = pd.read_csv(gold_file)
    all_cleaned_files.append({
        'dataset': 'Harga Emas',
        'file': 'harga_emas_cleaned.csv',
        'rows': len(df_summary),
        'columns': list(df_summary.columns),
        'date_range': f"{df_summary['tanggal'].min()} - {df_summary['tanggal'].max()}"
    })

# 2. Federal Funds Rate
import glob
ffr_files = glob.glob(os.path.join(base_path, "federal_funds_rate_cleaned_*.csv"))
if ffr_files:
    latest_ffr = max(ffr_files, key=os.path.getctime)
    df_summary = pd.read_csv(latest_ffr)
    all_cleaned_files.append({
        'dataset': 'Federal Funds Rate',
        'file': os.path.basename(latest_ffr),
        'rows': len(df_summary),
        'columns': list(df_summary.columns),
        'date_range': f"{df_summary['tanggal'].min()} - {df_summary['tanggal'].max()}"
    })

# 3. Jakarta Stock Exchange
jci_files = glob.glob(os.path.join(base_path, "jakarta_stock_exchange_cleaned_*.csv"))
if jci_files:
    latest_jci = max(jci_files, key=os.path.getctime)
    df_summary = pd.read_csv(latest_jci)
    all_cleaned_files.append({
        'dataset': 'Jakarta Stock Exchange',
        'file': os.path.basename(latest_jci),
        'rows': len(df_summary),
        'columns': list(df_summary.columns),
        'date_range': f"{df_summary['tanggal'].min()} - {df_summary['tanggal'].max()}"
    })

# 4. Data Inflasi
inflasi_files = glob.glob(os.path.join(base_path, "data_inflasi_cleaned_*.csv"))
if inflasi_files:
    latest_inflasi = max(inflasi_files, key=os.path.getctime)
    df_summary = pd.read_csv(latest_inflasi)
    all_cleaned_files.append({
        'dataset': 'Data Inflasi',
        'file': os.path.basename(latest_inflasi),
        'rows': len(df_summary),
        'columns': list(df_summary.columns),
        'date_range': f"{df_summary['tanggal'].min()} - {df_summary['tanggal'].max()}"
    })

# Tampilkan summary
print(f"\n📋 TOTAL DATASET YANG BERHASIL DI-CLEANING: {len(all_cleaned_files)}")
print()

for i, file_info in enumerate(all_cleaned_files, 1):
    print(f"{i}. 📊 {file_info['dataset']}")
    print(f"   📁 File: {file_info['file']}")
    print(f"   📏 Dimensi: {file_info['rows']} baris × {len(file_info['columns'])} kolom")
    print(f"   📅 Rentang tanggal: {file_info['date_range']}")
    print(f"   🏗️ Kolom: {', '.join(file_info['columns'])}")
    print()

print("="*60)
print("✅ SEMUA DATASET SIAP UNTUK ANALISIS EKONOMI!")
print()
print("🔍 Keempat dataset sudah dibersihkan dan siap untuk:")
print("   • Analisis korelasi multi-aset")
print("   • Time series forecasting")
print("   • Economic indicator dashboard")
print("   • Portfolio optimization")
print("   • Inflation hedging analysis")
print("   • Risk assessment modeling")
print()
print("📈 NEXT STEPS:")
print("   1. Merge datasets berdasarkan tanggal")
print("   2. Analisis korelasi antar variabel")
print("   3. Visualisasi trend historis")
print("   4. Pemodelan prediktif")
print("="*60)

🎉 PROSES CLEANING SEMUA DATA SELESAI! 🎉

📋 TOTAL DATASET YANG BERHASIL DI-CLEANING: 4

1. 📊 Harga Emas
   📁 File: harga_emas_cleaned.csv
   📏 Dimensi: 1899 baris × 7 kolom
   📅 Rentang tanggal: 2020-01-01 - 2025-03-13
   🏗️ Kolom: tanggal, usd_per_ounce, kurs_dollar_per_ounce, idr_per_ounce, usd_per_gram, kurs_dollar_per_gram, idr_per_gram

2. 📊 Federal Funds Rate
   📁 File: federal_funds_rate_cleaned_20250630_195749.csv
   📏 Dimensi: 1302 baris × 7 kolom
   📅 Rentang tanggal: 2019-12-31 - 2025-03-18
   🏗️ Kolom: tanggal, close_rate, open_rate, high_rate, low_rate, volume_billion, change_percent

3. 📊 Jakarta Stock Exchange
   📁 File: jakarta_stock_exchange_cleaned_20250702_095758.csv
   📏 Dimensi: 1262 baris × 7 kolom
   📅 Rentang tanggal: 2019-12-30 - 2025-03-13
   🏗️ Kolom: tanggal, close_index, open_index, high_index, low_index, volume_billion, change_percent

4. 📊 Data Inflasi
   📁 File: data_inflasi_cleaned_20250702_102841.csv
   📏 Dimensi: 63 baris × 2 kolom
   📅 Rentang tanggal